<a href="https://colab.research.google.com/github/theforces/YOLO-hand-detection/blob/main/MLAV_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title First try for .mat files
import scipy.io
mat = scipy.io.loadmat('/content/Buffy_1.mat')

print(mat.keys())
print('\n')
print(mat)

dict_keys(['__header__', '__version__', '__globals__', 'boxes'])


{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI, Created on: Mon Jul 18 17:41:20 2011', '__version__': '1.0', '__globals__': [], 'boxes': array([[array([[(array([[488.42954942, 345.62261032]]), array([[461.57045058, 348.37738968]]), array([[465.57045058, 387.37738968]]), array([[492.42954942, 384.62261032]]))]],
      dtype=[('a', 'O'), ('b', 'O'), ('c', 'O'), ('d', 'O')]),
        array([[(array([[449.81237318, 397.24488989]]), array([[462.18762682, 426.75511011]]), array([[493.18762682, 413.75511011]]), array([[480.81237318, 384.24488989]]))]],
      dtype=[('a', 'O'), ('b', 'O'), ('c', 'O'), ('d', 'O')])]],
      dtype=object)}


In [ ]:
#@title Default title text
scipy.io.whosmat('/content/Buffy_1.mat')

In [ ]:
#@title Convert from .mat to .csv
import scipy.io
import pandas as pd

mat = scipy.io.loadmat('/content/Buffy_1.mat')
mat = {k:v for k, v in mat.items() if k[0] != '_'}
data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()}) # compatible for both python 2.x and python 3.x

data.to_csv("example.csv")

In [ ]:
#@title Convert multiple files from .mat to .csv
import os
import scipy.io
import pandas as pd

for root, dirs, files in os.walk('/content/training_data/annotations/'):
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension == '.mat':
          # print(filename  + extension )
          # print( os.getcwd())
          mat = scipy.io.loadmat('/content/training_data/annotations/'+ filename + extension)
          mat = {k:v for k, v in mat.items() if k[0] != '_'}
          data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()}) # compatible for both python 2.x and python 3.x
          data.to_csv('/content/training_data/annotations_csv/' + filename + '.csv')


In [ ]:
!zip -r /content/annotations_csv.zip /content/training_data/annotations_csv/

In [ ]:
%cd /content/

/content


In [ ]:
#@title Experimenting with multiple files
# importing csv module
import csv
import numpy as np
import re
from ast import literal_eval
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


def is_float_string(string):
    try:
        float(string)
    except ValueError:
        return False
    return str(float(string)) == string


def convert(xmin, ymin, xmax, ymax, img_w, img_h):
    dw = 1./(img_w)
    dh = 1./(img_h)
    x = (xmin + xmax)/2.0 - 1
    y = (ymin + ymax)/2.0 - 1
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

heights = []
widths = []
src='/content/images/'
files = os.listdir(src)
for idx, image_src in enumerate(files):
  image = Image.open(f'{src}{image_src}',"r")  
  width, height = image.size
  widths.append(width)
  heights.append(height)
  print('width for Image ', image.filename, ' is ', width)
  print('height for Image ', image.filename, ' is ', height)
  
print(widths)
print(heights)

# with Image.open('/content/Skin_124.jpg') as img:
#     width, height = img.size
#     print('width= ',width)
#     print('height= ',height)

# csv file name
filename = "Skin_124.csv"
  
# initializing the titles and rows list
fields = []
rows = []
  
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
      
    # extracting field names through first row
    fields = next(csvreader)
  
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)
  
    # get total number of rows
    # print("Total no. of rows: %d"%(csvreader.line_num))
  
# printing the field names
# print('Field names are:' + ', '.join(field for field in fields))

columns_list = []

#  printing first 5 rows
#print('\nFirst 5 rows are:\n')
for row in rows[:5]:
    columns = []

    classes = int(row[0])
    coordonates = re.sub('[\array\()\[\]]', '', row[1])
    #print(coordonates)

    coordonates_array = coordonates.split(',')
    coordonates_array = [num.strip() for num in coordonates_array]
    coordonates_array = [ i for i in coordonates_array if is_float_string(i) ]
    new_coordonates_array = [float(i) for i in coordonates_array]
    coordonates_x = new_coordonates_array[0::2]
    coordonates_y = new_coordonates_array[1::2]

    xmin = float(min(coordonates_x))
    xmax = float(max(coordonates_x))
    ymin = float(min(coordonates_y))
    ymax = float(max(coordonates_y))

    x = xmin
    y = ymin
    width = abs(xmax - xmin)
    height = abs(ymax - ymin)

    x = xmin / width
    y = ymin / height
    w = abs((xmax - xmin) / width)
    h = abs((ymax - ymin) / height)

    # print('coordonates_array = ', coordonates_array)
    # print('coordonates_x = ', coordonates_x)
    # print('coordonates_y = ', coordonates_y)
    # print('xmin = ', xmin)
    # print('xmax = ', xmax)
    # print('ymin = ', ymin)
    # print('ymax = ', ymax)
    # print('x = ', x)
    # print('y = ', y)
    # print('w = ', w)
    # print('h = ', h)
    
    columns.append(int(classes))
    columns.append(x)
    columns.append(y)
    columns.append(w)
    columns.append(h)

    columns_list.append(columns)

    # print('columns = ', columns)
    # print('\n')
    
    data = pd.DataFrame(columns_list)
    data.to_csv('Skin_124' + '.txt', sep=' ', header=False, index=False, mode='w' )

    

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Files/hand_dataset/training_dataset/training_data/annotations_csv

/content/training_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/images

/content/images


In [ ]:
from PIL import Image
import os

old_heights = []
old_widths = []
old_filenames = []
src='/content/images/'
files = os.listdir(src)


for root, dirs, files in os.walk('/content/images/'):
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension == '.jpg':
          for idx, image_src in enumerate(files):
            image = Image.open(f'{src}{image_src}',"r")  
            width, height = image.size
            old_widths.append(width)
            old_heights.append(height)
            old_filenames.append(image.filename)
            print('width for Image ', image.filename, ' is ', width)
            print('height for Image ', image.filename, ' is ', height)

heights = list(set(heights))
widths = list(set(widths))
filenames = list(set(filenames))

print('widths =', widths)
print('heights =', heights)
print('filenames =', filenames)

width for Image  /content/images/Buffy_2.jpg  is  612
height for Image  /content/images/Buffy_2.jpg  is  704
width for Image  /content/images/Buffy_1.jpg  is  618
height for Image  /content/images/Buffy_1.jpg  is  707
width for Image  /content/images/Buffy_2.jpg  is  612
height for Image  /content/images/Buffy_2.jpg  is  704
width for Image  /content/images/Buffy_1.jpg  is  618
height for Image  /content/images/Buffy_1.jpg  is  707
widths = [618, 612]
heights = [704, 707]
filenames = ['/content/images/Buffy_2.jpg', '/content/images/Buffy_1.jpg']


In [ ]:
%cd /content/test/test_csv

/content/test/test_csv


In [ ]:
import os
import scipy.io
import pandas as pd
import csv
import numpy as np
import re
from ast import literal_eval
import itertools 

def is_float_string(string):
    try:
        float(string)
    except ValueError:
        return False
    return str(float(string)) == string


def convert(xmin, ymin, xmax, ymax, img_w, img_h):
    dw = 1./(img_w)
    dh = 1./(img_h)
    x = (xmin + xmax)/2.0 - 1
    y = (ymin + ymax)/2.0 - 1
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convertVersion2(xmin, ymin, xmax, ymax, img_w, img_h):
    width = img_w
    height = img_h
    x = (((xmin + xmax)/2)-1) / width
    y = (((ymin+ ymax)/2)-1) / height
    w = (xmax - xmin) / width
    h = (ymax - ymin) / height
    return (x,y,w,h)


for root, dirs, files in os.walk('/content/test/test_csv/'):
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension == '.csv':
          # print(filename  + extension )
          # print( os.getcwd())
          fields = []
          rows = []
            
          # reading csv file
          with open(filename+extension, 'r') as csvfile:
              # creating a csv reader object
              csvreader = csv.reader(csvfile)
                
              # extracting field names through first row
              fields = next(csvreader)
            
              # extracting each data row one by one
              for row in csvreader:
                  rows.append(row)
            
              # get total number of rows
              # print("Total no. of rows: %d"%(csvreader.line_num))
            
          # printing the field names
          # print('Field names are:' + ', '.join(field for field in fields))

          columns_list = []

          #  printing first 5 rows
          # print('\nFirst 5 rows are:\n')
          for (row,width,height) in zip(rows,widths,heights):
              columns = []

              classes = int(row[0])
              coordonates = re.sub('[\array\()\[\]]', '', row[1])
              # print(coordonates)

              coordonates_array = coordonates.split(',')
              coordonates_array = [num.strip() for num in coordonates_array]
              coordonates_array = [ i for i in coordonates_array if is_float_string(i) ]
              new_coordonates_array = [float(i) for i in coordonates_array]

              print(new_coordonates_array)
              coordonates_x = new_coordonates_array[0::2]
              coordonates_y = new_coordonates_array[1::2]

              xmin = float(min(coordonates_x))
              xmax = float(max(coordonates_x))
              ymin = float(min(coordonates_y))
              ymax = float(max(coordonates_y))

              x,y,w,h = convertVersion2(xmin,ymin,xmax,ymax,width,height)

              # x = xmin
              # y = ymin
              # width = abs(xmax - xmin)
              # height = abs(ymax - ymin)

              # x = xmin / width
              # y = ymin / height
              # w = abs((xmax - xmin) / width)
              # h = abs((ymax - ymin) / height)

              # print('coordonates_array = ', coordonates_array)
              # print('coordonates_x = ', coordonates_x)
              # print('coordonates_y = ', coordonates_y)
              # print('xmin = ', xmin)
              # print('xmax = ', xmax)
              # print('ymin = ', ymin)
              # print('ymax = ', ymax)
              # print('x = ', x)
              # print('y = ', y)
              # print('w = ', w)
              # print('h = ', h)
              
              columns.append(int(classes))
              columns.append(x)
              columns.append(y)
              columns.append(w)
              columns.append(h)

              columns_list.append(columns)

              # print('columns = ', columns)
              # print('\n')
              
              data = pd.DataFrame(columns_list)
              data.to_csv('/content/test/test_txt/' + filename + '.txt', sep=' ', header=False, index=False, mode='w' )

[488.42954942, 345.62261032, 461.57045058, 348.37738968, 465.57045058, 387.37738968, 492.42954942, 384.62261032]
[449.81237318, 397.24488989, 462.18762682, 426.75511011, 493.18762682, 413.75511011, 480.81237318, 384.24488989]
[480.96964147, 329.25321469, 455.03035853, 336.74678531, 468.03035853, 381.74678531, 493.96964147, 374.25321469]
[447.67487812, 388.91215466, 458.32512188, 419.08784534, 492.32512188, 407.08784534, 481.67487812, 376.91215466]


In [ ]:
!zip -r /content/test/test_txt.zip /content/test/test_txt/

In [ ]:
%cd /content/test/test_singular

/content/test/test_singular


In [ ]:
#@title Version 2 
import os
import scipy.io
import pandas as pd
import csv
import numpy as np
import re
from ast import literal_eval
import itertools 

def is_float_string(string):
    try:
        float(string)
    except ValueError:
        return False
    return str(float(string)) == string


def convert(xmin, ymin, xmax, ymax, img_w, img_h):
    dw = 1./(img_w)
    dh = 1./(img_h)
    x = (xmin + xmax)/2.0 - 1
    y = (ymin + ymax)/2.0 - 1
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def process_coordonates(eight_number_list):

  num_1_x = eight_number_list[0]
  num_1_y = eight_number_list[1]
  
  num_2_x = eight_number_list[2]
  num_2_y = eight_number_list[3]

  num_3_x = eight_number_list[4]
  num_3_y = eight_number_list[5]

  num_4_x = eight_number_list[6]
  num_4_y = eight_number_list[7]

  
  coordonates_dict1 = dict({num_1_x: num_1_y})
  coordonates_dict2 = dict({num_2_x: num_2_y})
  coordonates_dict3 = dict({num_3_x: num_3_y})
  coordonates_dict4 = dict({num_4_x: num_4_y})

  

for root, dirs, files in os.walk('/content/test/test_singular/'):
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension == '.csv':
          #print(filename  + extension )
          # print( os.getcwd())
          fields = []
          rows = []
            
          # reading csv file
          with open(filename+extension, 'r') as csvfile:
              # creating a csv reader object
              csvreader = csv.reader(csvfile)
                
              # extracting field names through first row
              fields = next(csvreader)
            
              # extracting each data row one by one
              for row in csvreader:
                  rows.append(row)
            
              # get total number of rows
              # print("Total no. of rows: %d"%(csvreader.line_num))
            
          # printing the field names
          # print('Field names are:' + ', '.join(field for field in fields))

          columns_list = []

          #  printing first 5 rows
          # print('\nFirst 5 rows are:\n')
          for (row) in rows[:5]:
              columns = []

              classes = int(row[0])
              coordonates = re.sub('[\array\()\[\]]', '', row[1])
              #print(coordonates)

              coordonates_array = coordonates.split(',')
              coordonates_array = [num.strip() for num in coordonates_array]
              coordonates_array = [ i for i in coordonates_array if is_float_string(i) ]
              new_coordonates_array = [float(i) for i in coordonates_array]

              
              process_coordonates(new_coordonates_array)

              coordonates_x = new_coordonates_array[0::2]
              coordonates_y = new_coordonates_array[1::2]

              xmin = float(min(coordonates_x))
              xmax = float(max(coordonates_x))
              ymin = float(min(coordonates_y))
              ymax = float(max(coordonates_y))

              x = xmin
              y = ymin
              width = abs(xmax - xmin)
              height = abs(ymax - ymin)

              x = xmin / width
              y = ymin / height
              w = abs((xmax - xmin) / width)
              h = abs((ymax - ymin) / height)

              # print('coordonates_array = ', coordonates_array)
              # print('coordonates_x = ', coordonates_x)
              # print('coordonates_y = ', coordonates_y)
              # print('xmin = ', xmin)
              # print('xmax = ', xmax)
              # print('ymin = ', ymin)
              # print('ymax = ', ymax)
              # print('x = ', x)
              # print('y = ', y)
              # print('w = ', w)
              # print('h = ', h)
              
              columns.append(int(classes))
              columns.append(x)
              columns.append(y)
              columns.append(w)
              columns.append(h)

              columns_list.append(columns)

              # print('columns = ', columns)
              # print('\n')
              
              data = pd.DataFrame(columns_list)
              data.to_csv('/content/test/test_singular_txt/' + filename + '.txt', sep=' ', header=False, index=False, mode='w' )
          

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/hands/downloads/hand_dataset.tar.gz

--2021-12-12 10:37:41--  https://www.robots.ox.ac.uk/~vgg/data/hands/downloads/hand_dataset.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250468306 (239M) [application/x-gzip]
Saving to: ‘hand_dataset.tar.gz’

hand_dataset.tar.gz 100%[===================>] 238.86M  30.2MB/s    in 8.8s    

2021-12-12 10:37:50 (27.3 MB/s) - ‘hand_dataset.tar.gz’ saved [250468306/250468306]



In [ ]:
!tar -xzvf "/content/hand_dataset.tar.gz" -C "/content"     

Streaming output truncated to the last 5000 lines.
hand_dataset/training_dataset/training_data/images/Inria_11.jpg
hand_dataset/training_dataset/training_data/images/Inria_110.jpg
hand_dataset/training_dataset/training_data/images/Inria_111.jpg
hand_dataset/training_dataset/training_data/images/Inria_112.jpg
hand_dataset/training_dataset/training_data/images/Inria_113.jpg
hand_dataset/training_dataset/training_data/images/Inria_114.jpg
hand_dataset/training_dataset/training_data/images/Inria_115.jpg
hand_dataset/training_dataset/training_data/images/Inria_116.jpg
hand_dataset/training_dataset/training_data/images/Inria_117.jpg
hand_dataset/training_dataset/training_data/images/Inria_118.jpg
hand_dataset/training_dataset/training_data/images/Inria_119.jpg
hand_dataset/training_dataset/training_data/images/Inria_12.jpg
hand_dataset/training_dataset/training_data/images/Inria_120.jpg
hand_dataset/training_dataset/training_data/images/Inria_121.jpg
hand_dataset/training_dataset/training_da

In [ ]:
!git clone https://github.com/SignusRobotics/oxford-hands-to-yolo.git

Cloning into 'oxford-hands-to-yolo'...
remote: Enumerating objects: 12, done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (12/12), done.


In [ ]:
!mv hand_dataset oxford-hands-to-yolo

In [ ]:
%cd /content/oxford-hands-to-yolo

/content/oxford-hands-to-yolo


In [ ]:
!python converter.py

else
It took 0:00:03.42 to execute this
